In [84]:
# titanic_train.csv 파일을 로딩합니다
#결측치를 처리합니다
    # - Age는 전체 인원의 나이의 평균값으로
    # - Cabin과 Embarked는 모두 'N'
# 'PassengerId','Name','Ticket'컬럼은 삭제
# 'Cabin','Sex','Embarked'에 대해서만 라벨 인코딩 실행

# 남은 컬럼은 Pclass, Sex, Age, SibSp, Parch, Fare, Cabin, Embarked
# 데이터를 train_test_split 함수를 이용하여 8:2 비율로 나눔
# 정확도를 계산하여 출력

In [85]:
# 각 모듈들 임포트
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
# K폴드 교차 검증을 위한 모듈 임포트
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# titanic_train.csv 파일을 로딩합니다
df = pd.read_csv('titanic_train.csv', encoding="cp949")

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [87]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [88]:
#결측치를 처리합니다
    # - Age는 전체 인원의 나이의 평균값으로
#df.groupby(["PassengerId"])["Age"].mean()  # fillna로 채우지 않았음
df["Age"].fillna( df["Age"].mean(), inplace=True)
    # - Cabin과 Embarked는 모두 'N'
df["Cabin"].fillna( "N", inplace=True)
df["Embarked"].fillna( "N", inplace=True)
# 'PassengerId','Name','Ticket'컬럼은 삭제
cols = ["PassengerId","Name","Ticket"]
titanic = df.drop(cols, axis=1)
titanic.head(150)
# 남은 컬럼은 Pclass, Sex, Age, SibSp, Parch, Fare, Cabin, Embarked

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,N,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,N,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,N,S
...,...,...,...,...,...,...,...,...,...
145,0,2,male,19.0,1,1,36.7500,N,S
146,1,3,male,27.0,0,0,7.7958,N,S
147,0,3,female,9.0,2,2,34.3750,N,S
148,0,2,male,36.5,0,2,26.0000,F2,S


In [89]:
titanic.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [90]:
# Cabin, Sex, Embarked에 대해서만 라벨인코딩을 실행합니다
# Cabin 데이터의 첫 글자를 제외한 나머지를 삭제합니다(첫 글자만 취합니다)
titanic['Cabin'] = titanic['Cabin'].str[:1]
titanic['Cabin'].value_counts()

N    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: Cabin, dtype: int64

In [91]:
print(titanic['Sex'].value_counts())

male      577
female    314
Name: Sex, dtype: int64


In [92]:
print(titanic['Embarked'].value_counts())

S    644
C    168
Q     77
N      2
Name: Embarked, dtype: int64


In [93]:
# 'Cabin','Sex','Embarked'에 대해서만 라벨 인코딩 실행
from sklearn import preprocessing
features = ['Cabin','Sex','Embarked']

# 아래 과정을 세번 거쳐서 하는 방법
#le = preprocessing.LabelEncoder()
#le = LabelEncoder()
#le = le.fit(titanic['Cabin'])
#titanic['Cabin'] = le.transform(titanic['Cabin'])

for f in features:
    le = preprocessing.LabelEncoder()
    le = le.fit(titanic[f])
    titanic[f] = le.transform( titanic[f] )
titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,1,0,7.2500,7,3
1,1,1,0,38.0,1,0,71.2833,2,0
2,1,3,0,26.0,0,0,7.9250,7,3
3,1,1,0,35.0,1,0,53.1000,2,3
4,0,3,1,35.0,0,0,8.0500,7,3


In [24]:
# 'Cabin','Sex','Embarked'에 대해서만 라벨 인코딩 실행
from sklearn.preprocessing import LabelEncoder
colums = ["Cabin","Sex","Embarked"]
encoder = LabelEncoder()
encoder.fit(colums)
labels = encoder.transform(colums)
print('인코딩 변환값:', labels)
# 데이터를 train_test_split 함수를 이용하여 8:2 비율로 나눔
# 정확도를 계산하여 출력

인코딩 변환값: [0 2 1]


In [94]:
# 데이터를 train_test_split 함수를 이용하여 8:2 비율로 나눔
# 타겟값을 데이터 셋에서 분리하여 y_titanic에 저장
y_titanic = titanic['Survived']
# 원본데이터에서 다겟값을 제외한 나머지 컬럼들로 X_titanic 구성
X_titanic = titanic.drop('Survived', axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_titanic, y_titanic, \
                                                   test_size=0.2, random_state=11) 
# 정확도를 계산하여 출력

In [96]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [98]:
# 결정트리, Random Forest. 로지스틱 회귀를 위한 사이킷런 Classifier 클래스 생성
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
lr_clf = LogisticRegression()

In [116]:
# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
acc = accuracy_score(y_test, dt_pred)
print('DecisionTreeClassifier accuracy : {0:.4f}'.format(acc))

DecisionTreeClassifier accuracy : 0.8101


In [129]:
# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
acc = accuracy_score(y_test, rf_pred)
print('RandomForestClassifier accuracy : {0:.4f}'.format(acc))

RandomForestClassifier accuracy : 0.8492


In [130]:
# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
acc = accuracy_score(y_test, lr_pred)
print('LogisticRegression accuracy : {0:.4f}'.format(acc))

LogisticRegression accuracy : 0.8492


C:\Users\JAVA01\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [118]:
X_test[0:1]

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
431,3,0,29.699118,1,0,16.1,7,3


In [119]:
dt_clf.predict(X_test[0:1])

array([1], dtype=int64)

In [131]:
rf_clf.predict(X_test[0:1])

array([1], dtype=int64)

In [139]:
lr_clf.predict(X_test[0:1])

array([1], dtype=int64)

In [144]:
# logistic Regression 모델이 예측한 결과들은 1열모양으로 재배치
lr_pred_reshape = lr_pred.reshape(-1, 1)

In [145]:
X_test['survived_lr'] = lr_pred_reshape
X_test['survived_lr'] = X_test['survived_lr'].apply(lambda x : '생존' if x==1 else '사망')

<ipython-input-145-05bd397739c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['survived_lr'] = lr_pred_reshape
<ipython-input-145-05bd397739c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['survived_lr'] = X_test['survived_lr'].apply(lambda x : '생존' if x==1 else '사망')


In [146]:
X_test.drop(['survived_lr'], axis=1, inplace=True)
X_test

C:\Users\JAVA01\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
431,3,0,29.699118,1,0,16.1000,7,3
821,3,1,27.000000,0,0,8.6625,7,3
629,3,1,29.699118,0,0,7.7333,7,2
626,2,1,57.000000,0,0,12.3500,7,2
665,2,1,32.000000,2,0,73.5000,7,3
...,...,...,...,...,...,...,...,...
638,3,0,41.000000,0,5,39.6875,7,3
771,3,1,48.000000,0,0,7.8542,7,3
521,3,1,22.000000,0,0,7.8958,7,3
711,1,1,29.699118,0,0,26.5500,2,3


In [135]:
from sklearn.model_selection import KFold
# 폴드 세트를 5개로하여 각 세트마다의 정확도와 평균 정확도를 출력하세요
kfold = KFold(n_splits=5)
cv_accuracy = []

In [168]:
n=1
for train_index, test_index in kfold.split(X_train):
    print('split #',n)
    print('train index :\n',X_train)
    print('test index :\n', y_test)
    n = n+1
    print()

split # 1
train index :
      Pclass  Sex   Age  SibSp  Parch      Fare  Cabin  Embarked
333       3    1  16.0      2      0   18.0000      7         3
662       1    1  47.0      0      0   25.5875      4         3
382       3    1  32.0      0      0    7.9250      7         3
331       1    1  45.5      0      0   28.5000      2         3
149       2    1  42.0      0      0   13.0000      7         3
..      ...  ...   ...    ...    ...       ...    ...       ...
269       1    0  35.0      0      0  135.6333      2         3
337       1    0  41.0      0      0  134.5000      4         0
91        3    1  20.0      0      0    7.8542      7         3
80        3    1  22.0      0      0    9.0000      7         3
703       3    1  25.0      0      0    7.7417      7         2

[712 rows x 8 columns]
test index :
 0      0
1      1
2      1
3      1
4      0
      ..
138    0
139    0
140    0
141    1
142    1
Name: Survived, Length: 143, dtype: int64

split # 2
train index :
   

In [169]:
cnt = 0
for train_index, test_index in kfold.split(X_titanic):
    X_train, X_test = X_titanic.values[train_index], X_titanic.values[test_index]
    y_train, y_test = y_titanic.values[train_index], y_titanic.values[test_index]
    dt_clf.fit(X_train, y_train)  # 학습
    predictions = dt_clf.predict(X_test)  # 테스트예측
    # 실제데이터와 예측데이터로 정확도 계산
    accuracy = accuracy_score(y_test, predictions)
    cnt = cnt + 1
    # shape의 결과(행 개수, 열 개수)에서 [0] 데이터(행 개수)로 각 데이터 수 추출
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    # 정확도, 학습데이터 수, 테스트데이터 수 순으로 출력
    print('#{0}번째 k폴드의 정확도 : {1:.3f}, 학습데이터 개수 : {2},\
        테스트데이터개수 : {3}'.format(cnt, accuracy, train_size, test_size))
    # 다섯개의 정확도의 평균을 계산하기 위해 정확도를 리스트에 저장
    cv_accuracy.append(accuracy)
# 개별 iterration별 정확도를 합하여 평균 정확도 계산
print('## 평균 검증 정확도 : ', np.mean(cv_accuracy))


#1번째 k폴드의 정확도 : 0.749, 학습데이터 개수 : 712,        테스트데이터개수 : 179
#2번째 k폴드의 정확도 : 0.781, 학습데이터 개수 : 713,        테스트데이터개수 : 178
#3번째 k폴드의 정확도 : 0.781, 학습데이터 개수 : 713,        테스트데이터개수 : 178
#4번째 k폴드의 정확도 : 0.770, 학습데이터 개수 : 713,        테스트데이터개수 : 178
#5번째 k폴드의 정확도 : 0.843, 학습데이터 개수 : 713,        테스트데이터개수 : 178
## 평균 검증 정확도 :  0.7845521310652188
